## 88. Funciones de ventana:

In [1]:
%load_ext autoreload
%autoreload 2

import os
path = os.environ.get("HOME")
os.chdir(f"{path}/github/pyspark-hand-on")
from src.utils.utils import spark_session

spark = spark_session()

25/11/06 21:36:37 WARN Utils: Your hostname, personal-computer resolves to a loopback address: 127.0.1.1; using 172.17.0.1 instead (on interface docker0)
25/11/06 21:36:37 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/11/06 21:36:38 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
df = spark.read.parquet("./data/funciones_ventana.parquet")
df.printSchema()

root
 |-- nombre: string (nullable = true)
 |-- edad: integer (nullable = true)
 |-- departamento: string (nullable = true)
 |-- evaluacion: integer (nullable = true)



In [3]:
df.show(truncate=False)

+-------+----+------------+----------+
|nombre |edad|departamento|evaluacion|
+-------+----+------------+----------+
|Lazaro |45  |letras      |98        |
|Raul   |24  |matemática  |76        |
|Maria  |34  |matemática  |27        |
|Jose   |30  |química     |78        |
|Susana |51  |química     |98        |
|Juan   |44  |letras      |89        |
|Julia  |55  |letras      |92        |
|Kadir  |38  |arquitectura|39        |
|Lilian |23  |arquitectura|94        |
|Rosa   |26  |letras      |91        |
|Aian   |50  |matemática  |73        |
|Yaneisy|29  |letras      |89        |
|Enrique|40  |química     |92        |
|Jon    |25  |arquitectura|78        |
|Luisa  |39  |arquitectura|94        |
+-------+----+------------+----------+



In [4]:
from pyspark.sql.functions import col, desc, row_number, rank, dense_rank
from pyspark.sql.window import Window

In [5]:
WindowSpec =  Window.partitionBy("departamento").orderBy(desc("evaluacion"))

In [6]:
## row_number
df.withColumn("row_number", row_number().over(WindowSpec)).filter(col("row_number").isin(1,2)).show()

+-------+----+------------+----------+----------+
| nombre|edad|departamento|evaluacion|row_number|
+-------+----+------------+----------+----------+
| Lilian|  23|arquitectura|        94|         1|
|  Luisa|  39|arquitectura|        94|         2|
| Lazaro|  45|      letras|        98|         1|
|  Julia|  55|      letras|        92|         2|
|   Raul|  24|  matemática|        76|         1|
|   Aian|  50|  matemática|        73|         2|
| Susana|  51|     química|        98|         1|
|Enrique|  40|     química|        92|         2|
+-------+----+------------+----------+----------+



In [7]:
#### rank
df.withColumn("rank", rank().over(WindowSpec)).show()

+-------+----+------------+----------+----+
| nombre|edad|departamento|evaluacion|rank|
+-------+----+------------+----------+----+
| Lilian|  23|arquitectura|        94|   1|
|  Luisa|  39|arquitectura|        94|   1|
|    Jon|  25|arquitectura|        78|   3|
|  Kadir|  38|arquitectura|        39|   4|
| Lazaro|  45|      letras|        98|   1|
|  Julia|  55|      letras|        92|   2|
|   Rosa|  26|      letras|        91|   3|
|   Juan|  44|      letras|        89|   4|
|Yaneisy|  29|      letras|        89|   4|
|   Raul|  24|  matemática|        76|   1|
|   Aian|  50|  matemática|        73|   2|
|  Maria|  34|  matemática|        27|   3|
| Susana|  51|     química|        98|   1|
|Enrique|  40|     química|        92|   2|
|   Jose|  30|     química|        78|   3|
+-------+----+------------+----------+----+



In [8]:
### dense_rank
df.withColumn("dense_rank", dense_rank().over(WindowSpec)).show()

+-------+----+------------+----------+----------+
| nombre|edad|departamento|evaluacion|dense_rank|
+-------+----+------------+----------+----------+
| Lilian|  23|arquitectura|        94|         1|
|  Luisa|  39|arquitectura|        94|         1|
|    Jon|  25|arquitectura|        78|         2|
|  Kadir|  38|arquitectura|        39|         3|
| Lazaro|  45|      letras|        98|         1|
|  Julia|  55|      letras|        92|         2|
|   Rosa|  26|      letras|        91|         3|
|   Juan|  44|      letras|        89|         4|
|Yaneisy|  29|      letras|        89|         4|
|   Raul|  24|  matemática|        76|         1|
|   Aian|  50|  matemática|        73|         2|
|  Maria|  34|  matemática|        27|         3|
| Susana|  51|     química|        98|         1|
|Enrique|  40|     química|        92|         2|
|   Jose|  30|     química|        78|         3|
+-------+----+------------+----------+----------+



In [9]:
## agregaciones con funciones de ventana:
windowSpecAgg = Window.partitionBy("departamento")

In [10]:
from pyspark.sql.functions import min, max, avg

In [12]:
(df.withColumn("min", min(col("evaluacion")).over(windowSpecAgg))
.withColumn("max", max(col("evaluacion")).over(windowSpecAgg))
.withColumn("avg", avg(col("evaluacion")).over(windowSpecAgg))
.withColumn("row_number", row_number().over(WindowSpec))
).show()

+-------+----+------------+----------+---+---+------------------+----------+
| nombre|edad|departamento|evaluacion|min|max|               avg|row_number|
+-------+----+------------+----------+---+---+------------------+----------+
| Lilian|  23|arquitectura|        94| 39| 94|             76.25|         1|
|  Luisa|  39|arquitectura|        94| 39| 94|             76.25|         2|
|    Jon|  25|arquitectura|        78| 39| 94|             76.25|         3|
|  Kadir|  38|arquitectura|        39| 39| 94|             76.25|         4|
| Lazaro|  45|      letras|        98| 89| 98|              91.8|         1|
|  Julia|  55|      letras|        92| 89| 98|              91.8|         2|
|   Rosa|  26|      letras|        91| 89| 98|              91.8|         3|
|   Juan|  44|      letras|        89| 89| 98|              91.8|         4|
|Yaneisy|  29|      letras|        89| 89| 98|              91.8|         5|
|   Raul|  24|  matemática|        76| 27| 76|58.666666666666664|         1|